In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import scipy.stats as stats
import numpy as np

In [15]:
df_BM= pd.read_csv('../Classifications/Join Collection NSC2_BND_M20.csv')
df_BM[:2]

C:\Users\5-20\AppData\Local\Temp\ipykernel_19320\2365204107.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_BM= pd.read_csv('../Classifications/Join Collection NSC2_BND_M20.csv')


,Unnamed: 0,EDC_SBA,BTH_YYYY,DTH_YYYYMM,MCARE_SUBJ_CD,OPRTN_YN,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,STD_YYYY
0,0,3000,1921LE,200707.0,14,0,3,5.0,2002
1,1,4950,1921LE,200707.0,13,0,2,5.0,2002


In [8]:
df_BM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137163 entries, 0 to 137162
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       137163 non-null  int64  
 1   EDC_SBA          137163 non-null  int64  
 2   BTH_YYYY         137163 non-null  object 
 3   DTH_YYYYMM       10269 non-null   float64
 4   MCARE_SUBJ_CD    137163 non-null  int64  
 5   OPRTN_YN         137163 non-null  int64  
 6   TOT_PRSC_DD_CNT  137163 non-null  int64  
 7   MCARE_RSLT_TYPE  133958 non-null  float64
 8   STD_YYYY         137163 non-null  int64  
dtypes: float64(2), int64(6), object(1)
memory usage: 9.4+ MB


In [16]:
### 지도학습 

#### 분류 - 이진분류(2개의 항목)

#### 과정
- 전처리 - 정형화 - 모델 교육 후 - 평가 서비스

In [17]:
### 숫자가 안들어간 것 
## survived name sex ticket 등 

In [19]:
import pymongo as mg
import pandas as pd
client = mg.MongoClient(host='mongodb://localhost:27017')

In [21]:
database = client['db_NHIS']
collection = database['Join Collection NSC2_BND_M20']

In [27]:
cursor = collection.find({})
list_BND_M20 = list(cursor)
list_BND_M20[:10]

[{'_id': ObjectId('64e6cbf07574768bae8e9ed1'),
  'RN_INDI': 294364,
  'BTH_YYYY': '1921LE',
  'DTH_YYYYMM': 200707,
  'COD1': 'T08-T14',
  'COD2': 'W00-W19',
  'RN_KEY': 2002080298703,
  'RN_INST': 26452,
  'MDCARE_STRT_DT': 20020830,
  'FORM_CD': 3,
  'MCARE_SUBJ_CD': 14,
  'SICK_SYM1': 'L028',
  'OFIJ_TYPE': 0,
  'OPRTN_YN': 0,
  'MDCARE_DD_CNT': 1,
  'VSHSP_DD_CNT': 1,
  'TOT_PRSC_DD_CNT': 3,
  'MCARE_RSLT_TYPE': 5,
  'EDC_ADD_RT': 0.15,
  'ED_RC_TOT_AMT': 10590,
  'EDC_SBA': 3000,
  'EDC_INSUR_BRDN_AMT': 7590,
  'STD_YYYY': 2002},
 {'_id': ObjectId('64e6cbf07574768bae8e9ed2'),
  'RN_INDI': 294364,
  'BTH_YYYY': '1921LE',
  'DTH_YYYYMM': 200707,
  'COD1': 'T08-T14',
  'COD2': 'W00-W19',
  'RN_KEY': 2002090466395,
  'RN_INST': 63715,
  'MDCARE_STRT_DT': 20020910,
  'FORM_CD': 3,
  'MCARE_SUBJ_CD': 13,
  'SICK_SYM1': 'J030',
  'OPRTN_YN': 0,
  'MDCARE_DD_CNT': 1,
  'VSHSP_DD_CNT': 1,
  'TOT_PRSC_DD_CNT': 2,
  'MCARE_RSLT_TYPE': 5,
  'EDC_ADD_RT': 0.15,
  'ED_RC_TOT_AMT': 16520,
  'EDC

In [29]:
df_BND_M20 = pd.DataFrame(list_BND_M20)
df_BND_M20[:5]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,64e6cbf07574768bae8e9ed1,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,0,1,1,3,5.0,0.15,10590,3000,7590,2002
1,64e6cbf07574768bae8e9ed2,294364,1921LE,200707.0,T08-T14,W00-W19,2002090466395,63715,20020910,3,...,0,1,1,2,5.0,0.15,16520,4950,11570,2002
2,64e6cbf07574768bae8e9ed3,294364,1921LE,200707.0,T08-T14,W00-W19,2002091053122,63715,20020909,3,...,0,3,3,3,5.0,0.15,27620,9000,18620,2002
3,64e6cbf07574768bae8e9ed4,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,0,1,1,3,5.0,0.15,12920,3000,9920,2002
4,64e6cbf07574768bae8e9ed5,294364,1921LE,200707.0,T08-T14,W00-W19,2002060822342,23506,20020628,3,...,0,1,1,0,5.0,0.15,14230,3000,11230,2002


In [31]:
df_BND_M20['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [39]:
len('F_') , len('J209') ,'J209'[:3], str('T18')[:3]

(2, 4, 'J20', 'T18')

In [37]:
def convertSICK_SYM1(sick_sym) :
    if len (sick_sym) < 3:
        return None
    else :
        return sick_sym[:3]

In [40]:
convertSICK_SYM1('J209'),convertSICK_SYM1('F_')

('J20', None)

In [42]:
df_BND_M20['SICK_SYM_3'] = df_BND_M20['SICK_SYM1'].apply(convertSICK_SYM1)

In [44]:
df_BND_M20['SICK_SYM_3'].value_counts()

J20    8917
I10    8427
J03    4187
M54    3904
J06    3460
       ... 
G97       1
K01       1
S28       1
I72       1
P28       1
Name: SICK_SYM_3, Length: 747, dtype: int64

In [46]:
df_BND_M20_dropna = df_BND_M20.dropna()
df_BND_M20_dropna.isnull().sum()

_id                   0
RN_INDI               0
BTH_YYYY              0
DTH_YYYYMM            0
COD1                  0
COD2                  0
RN_KEY                0
RN_INST               0
MDCARE_STRT_DT        0
FORM_CD               0
MCARE_SUBJ_CD         0
SICK_SYM1             0
OFIJ_TYPE             0
OPRTN_YN              0
MDCARE_DD_CNT         0
VSHSP_DD_CNT          0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE       0
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
EDC_INSUR_BRDN_AMT    0
STD_YYYY              0
SICK_SYM_3            0
dtype: int64

In [47]:
df_BND_M20_dropna[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM_3
0,64e6cbf07574768bae8e9ed1,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,1,3,5.0,0.15,10590,3000,7590,2002,L02
3,64e6cbf07574768bae8e9ed4,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,1,3,5.0,0.15,12920,3000,9920,2002,K29


In [62]:
target_list = df_BND_M20_dropna['SICK_SYM_3'].value_counts().index.to_list()

In [64]:
target_list.index('L02'), target_list.index('J03')

(83, 6)

In [70]:
def applyIndexNumber(sick_sym_3):
    indexNumber = target_list.index(sick_sym_3)
    return indexNumber

In [71]:
df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)
df_BND_M20_dropna[:2]

KeyError: 'SICK_SYM1_3'